<a href="https://colab.research.google.com/github/athibhash5/myworkouts/blob/main/RNNexamples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import pandas as pd
#data=pd.read_csv('/content/train.csv',encoding='ISO-8859-1',error_bad_lines=False)
data = pd.read_csv('/content/train.csv', encoding='ISO-8859-1', skiprows=[5685])
data.head()

,User_ID,Description,Browser_Used,Device_Used,Is_Response
0,id10326,The room was kind of clean but had a VERY stro...,Edge,Mobile,not happy
1,id10327,I stayed at the Crown Plaza April -- - April -...,Internet Explorer,Mobile,not happy
2,id10328,I booked this hotel through Hotwire at the low...,Mozilla,Tablet,not happy
3,id10329,Stayed here with husband and sons on the way t...,InternetExplorer,Desktop,happy
4,id10330,My girlfriends and I stayed here to celebrate ...,Edge,Tablet,not happy


In [9]:
data.drop(['Browser_Used','Device_Used','User_ID'],axis=1,inplace=True)
data

,Description,Is_Response
0,The room was kind of clean but had a VERY stro...,0
1,I stayed at the Crown Plaza April -- - April -...,0
2,I booked this hotel through Hotwire at the low...,0
3,Stayed here with husband and sons on the way t...,1
4,My girlfriends and I stayed here to celebrate ...,0
...,...,...
5679,Love the Clift& very stylish hotel. Service is...,1
5680,I got upgraded to Hilton Financial SF through ...,1
5681,"Steps away from Petco Field, walking distance ...",1
5682,We stayed at the Beacon for seven nights in Ap...,0


In [10]:
def change(message):
    if message=="not happy":
        return 0
    else:
        return 1
data["Is_Response"]=data[['Is_Response']].applymap(change)
data

,Description,Is_Response
0,The room was kind of clean but had a VERY stro...,1
1,I stayed at the Crown Plaza April -- - April -...,1
2,I booked this hotel through Hotwire at the low...,1
3,Stayed here with husband and sons on the way t...,1
4,My girlfriends and I stayed here to celebrate ...,1
...,...,...
5679,Love the Clift& very stylish hotel. Service is...,1
5680,I got upgraded to Hilton Financial SF through ...,1
5681,"Steps away from Petco Field, walking distance ...",1
5682,We stayed at the Beacon for seven nights in Ap...,1


In [ ]:
# If you're working with a single column and prefer a Series, you can use data["Is_Response"]. If you want to ensure that the result is a DataFrame, even when selecting a single column, you can use data[['Is_Response']].

In [11]:
import nltk          #Natural Language Toolkit (NLTK) is a powerful Python library for working with human language data
nltk.download('omw-1.4')
nltk.download('punkt')
nltk.download('wordnet')


[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [12]:
import re           #re:Regular expressions are powerful tools for pattern matching and string manipulation. They allow you to search for and manipulate text based on patterns, making them particularly useful for tasks like string matching, validation, and data extraction.
nltk.download('stopwords')
from nltk.corpus import stopwords
st=stopwords.words('english')



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [13]:
data['Description']

0       The room was kind of clean but had a VERY stro...
1       I stayed at the Crown Plaza April -- - April -...
2       I booked this hotel through Hotwire at the low...
3       Stayed here with husband and sons on the way t...
4       My girlfriends and I stayed here to celebrate ...
                              ...                        
5679    Love the Clift& very stylish hotel. Service is...
5680    I got upgraded to Hilton Financial SF through ...
5681    Steps away from Petco Field, walking distance ...
5682    We stayed at the Beacon for seven nights in Ap...
5683    The best part of this hotel is the location. I...
Name: Description, Length: 5684, dtype: object

In [18]:
from nltk.stem import WordNetLemmatizer
lem=WordNetLemmatizer()
wd=[]
for i in data['Description']:
  if isinstance(i, str):
    sent=re.sub('[^A-Za_z]'," ",i)
    words=nltk.word_tokenize(sent)
    unique_words=set()
    for w in words:
       if w not in st:
        lw=lem.lemmatize(w)
        unique_words.add(lw)
    clean_statment=" ".join(unique_words)
    wd.append(clean_statment)
  else:
    wd.append(" ")


In [19]:
len(wd)

5684

In [20]:
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [23]:

size=50000                   #set the vocabulary size
msg=[]
for i in wd:
    oh=one_hot(i,size)       # One-hot encode each word in the vocabulary,
    msg.append(oh)
y=data['Is_Response']
padding_len=30
x=pad_sequences(msg,padding='post',maxlen=padding_len)
x[11]


array([26783,  8172, 42996, 44172, 15548, 49056, 26783, 18378, 12841,
       26678, 41382,  7171,  1393, 34602,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0], dtype=int32)

In [ ]:
#applying padding to your sequences using Keras' pad_sequences function. This is a common step when working with sequences of variable lengths, ensuring that all sequences have the same length
#The padding parameter is set to 'post', indicating that padding will be added at the end of each sequence.
#The maxlen parameter is set to padding_len, specifying the maximum length of the padded sequences.


In [24]:
from tensorflow.keras.layers import LSTM,Dense,Embedding
from tensorflow.keras import Sequential
features=50
model=Sequential()
model.add(Embedding(size,features,input_length=padding_len))                              # Add an Embedding layer
model.add(LSTM(units=50,input_length=(30,50),activation='relu',return_sequences=True))    # Add the first LSTM layer
model.add(LSTM(units=100,activation='relu'))                                              # Add the 2nd LSTM layer
model.add(Dense(1,activation='sigmoid'))                                                  # Add a Dense layer for the output with sigmoid activation

model.compile(optimizer='adam',loss='mean_absolute_error',metrics=['accuracy'])
model.fit(x,y,epochs=5)

Epoch 1/5
178/178 [==============================] - 8s 32ms/step - loss: 0.0388 - accuracy: 1.0000
Epoch 2/5
178/178 [==============================] - 6s 35ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 3/5
178/178 [==============================] - 6s 32ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 4/5
178/178 [==============================] - 6s 36ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 5/5
178/178 [==============================] - 6s 34ms/step - loss: 0.0000e+00 - accuracy: 1.0000


In [ ]:
#adds an Embedding layer to the model. The Embedding layer is often used for text data. It turns positive integers (indexes) into dense vectors of fixed size. In your case:
#input_length=padding_len: The length of input sequences, i.e., the maximum length after padding.
#adds the first LSTM (Long Short-Term Memory) layer to the model. LSTM is a type of recurrent neural network (RNN) layer that is capable of learning long-term dependencies. In your case:
#units=50: The number of LSTM units (neurons) in the layer.
#activation='relu': Rectified Linear Unit (ReLU) activation function is used.
#return_sequences=True: Indicates that the layer should return the full sequence of outputs for each input sequence.


In [25]:
x.shape

(5684, 30)

In [26]:
y.shape

(5684,)

In [27]:
predicts=['consider smell better stay kind overnight price Breakfast clean strong right room VERY average dog fussy staying Generally nothing ok free The Would']



In [50]:
predicts=['I stayed in Nikko on a business trip for a week or so and had a lovely stay. The location was great, lots of nice restaurants nearby and it was easy to walk everywhere.The room was clean and spacious, nothing too different or special though. The pool and gym were nice, but you had to pay some extra to get there. I had some issues with the Internet connection but everything was solved quickly. Great service!' ]

In [54]:
pred_data=[]
for i in predicts:
  res=one_hot(i,100)
  pred_data.append(res)
pred_data


[[80,
  81,
  90,
  81,
  25,
  73,
  11,
  40,
  24,
  73,
  7,
  21,
  94,
  44,
  36,
  73,
  35,
  80,
  67,
  28,
  91,
  19,
  70,
  89,
  3,
  63,
  9,
  44,
  83,
  91,
  49,
  64,
  62,
  93,
  67,
  90,
  91,
  13,
  44,
  81,
  37,
  93,
  94,
  21,
  93,
  67,
  67,
  10,
  44,
  2,
  27,
  3,
  67,
  85,
  36,
  64,
  9,
  71,
  2,
  64,
  22,
  80,
  80,
  36,
  71,
  96,
  39,
  67,
  75,
  42,
  67,
  27,
  91,
  33,
  84,
  19,
  23]]

In [59]:
from keras.preprocessing.sequence import pad_sequences
padded_pred=pad_sequences(pred_data,maxlen=30,padding='pre')
result=model.predict(padded_pred)
print(result)



1/1 [==============================] - 0s 17ms/step
[[1.]]


In [60]:
import numpy as np
np.argmax(result,axis=1)

array([0])

In [41]:
# array([0]): not happy